In [1]:
import sys
import torch
from torch import nn
import os

sys.path.append(os.path.abspath("../../data"))

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 1

/Users/vik/.virtualenvs/nnets/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from csv_data import WeatherDatasetWrapper

class WeatherDataset(WeatherDatasetWrapper):
    predictors = ["tmax", "tmin", "rain"]
    target = "tmax_tomorrow"
    sequence_length = 7

wrapper = WeatherDataset()
datasets = wrapper.generate_datasets(BATCH_SIZE)

In [3]:
train = datasets["train"]
valid = datasets["validation"]
test = datasets["test"]

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.rnn = nn.RNN(3, 4, 1, batch_first=True)
        self.dense = nn.Linear(4, 1)

    def forward(self, x):
        x, hidden = self.rnn(x)
        x = self.dense(x)
        return x

In [6]:
from statistics import mean

model = NeuralNetwork().to(DEVICE)
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

EPOCHS = 50
for epoch in range(EPOCHS):
    for batch, (sequence, target) in enumerate(train):
        optimizer.zero_grad()

        sequence = sequence.to(DEVICE)
        pred = model(sequence)

        loss = loss_fn(pred, target)
        loss.backward()
        optimizer.step()

    losses = []
    with torch.no_grad():
        for batch, (sequence, target) in enumerate(valid):
            sequence = sequence.to(DEVICE)
            pred = model(sequence)
            loss = loss_fn(pred, target)
            losses.append(loss.item())

    print(f"Epoch {epoch} valid loss: {mean(losses)}")

Epoch 0 valid loss: 33.17428879997056
Epoch 1 valid loss: 23.60488214924998
Epoch 2 valid loss: 22.19114640177241
Epoch 3 valid loss: 22.400515992742413
Epoch 4 valid loss: 21.346459689029317
Epoch 5 valid loss: 21.198199933534095
Epoch 6 valid loss: 21.295649544297614
Epoch 7 valid loss: 20.989147545589912
Epoch 8 valid loss: 20.836097326710888
Epoch 9 valid loss: 21.914816398795374
Epoch 10 valid loss: 21.209869512565778


KeyboardInterrupt: 